# Alunos: Keiya Nishio e Karen Nishio

# Projeto Web Scrapper

## Descrição
Este projeto tem como objetivo realizar um Web Scraping na página dos top 250 melhores filmes de acordo com o IMDB, uma das fontes mais confiáveis para avaliação cinematográfica. Utilizando técnicas de Web Scraping, os dados essenciais sobre cada filme, como título, ano de lançamento, classificação, nota de avaliação e duração, serão extraídos de forma automatizada.

Uma vez que os dados são coletados, eles serão submetidos a uma análise. Serão criados gráficos e visualizações que permitirão uma compreensão mais clara das tendências e padrões dentro da lista dos melhores filmes.

## Ferramentas
- Beautiful Soup
- Pandas

## Página IMDB
- https://m.imdb.com/chart/top/


## Instalando as o Beautiful Soup

In [ ]:
!pip install -U BeautifulSoup4

## Importanto as bibliotecas

In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import csv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Definindo o link do site para obter os dados

In [ ]:
url = 'https://m.imdb.com/chart/top/'

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'}

response = requests.get(url, headers=headers)


## Coleta de dados

### Títulos dos filmes

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
movies_titles = soup.find_all('h3', class_='ipc-title__text')
movies = []
for movie in movies_titles:
    movie_text = movie.get_text().replace('\n', "").strip(" ")
    if movie_text[0].isdigit():
        movies.append(movie_text)
movies

In [ ]:
movies = [movie.split('. ')[1] for movie in movies]

### As notas de avaliações de cada filme

In [ ]:
rating_span = soup.find_all('span', class_='ipc-rating-star--imdb')
ratings_list = []

for span in rating_span:
    rating_text = span.get_text()
    rating_match = re.search(r'(\d+\.\d+)', rating_text)
    if rating_match:
        rating = rating_match.group(0)
        ratings_list.append(rating)
    else:
        print("Nota do IMDb não encontrada")

if not ratings_list:
    print("Elemento de avaliação IMDb não encontrado")

print(ratings_list)

### Ano do lançamento

In [ ]:
years = soup.find_all('span', class_='sc-b189961a-8 kLaxqf cli-title-metadata-item')
year_list = []

for year in years:
    year_text = year.get_text()
    year_match = re.search(r'(\d+\d+\d+\d+)', year_text)
    if year_match:
        ano = year_match.group(0)
        year_list.append(ano)

year_list

## Duração dos filmes

In [ ]:
durations = soup.find_all('span', class_='sc-b189961a-8 kLaxqf cli-title-metadata-item')
duration_list = []

for duration in durations:
    duration_text = duration.get_text()
    duration_match = re.search(r'(\d+h \d+m)', duration_text)
    duration_match_hour = re.search(r'(\d+h)', duration_text)
    duration_match_min = re.search(r'(\d+m)', duration_text)
    if duration_match:
        duracao = duration_match.group(0)
        duration_list.append(duracao)
    elif duration_match_hour:
        duracao = duration_match_hour.group(0)
        duration_list.append(duracao)
    elif duration_match_min:
        duracao = duration_match_min.group(0)
        duration_list.append(duracao)

duration_list

In [ ]:
def convert_to_min(duration):
    if "h" not in duration:
        duration = duration.replace('m', '')
        return int(duration)

    duration = duration.replace(' ', '')
    hours = re.search(r'(\d+h)', duration)
    hours = int(hours.group(0).replace('h', '')) if hours else 0
    minutes = re.search(r'(\d+m)', duration)
    minutes = int(minutes.group(0).replace('m', '')) if minutes else 0
    minutes += hours * 60
    return minutes



duration_list_min = [convert_to_min(duration) for duration in duration_list]

duration_list_min

### Classificação

In [ ]:
classifications = soup.find_all('span', class_='sc-b189961a-8 kLaxqf cli-title-metadata-item')
cls_list = []

for cls in classifications:
    cls_text = cls.get_text()
    cls_match = re.search(r'^[A-Za-z]*[A-Za-z][A-Za-z0-9-. _]*$', cls_text)
    if cls_match:
        classification = cls_match.group(0)
        cls_list.append(classification)

# Adiciona classificação em filmes sem classificação
cls_list.insert(57, "PG")
cls_list.insert(77, "R")
cls_list.insert(83, "PG-13")
cls_list

## Criando o banco de dados

In [ ]:
df = pd.DataFrame()
df['Title'] = movies
df['Ratings'] = ratings_list
df['Year'] = year_list
df['Duration (Minutes)'] = duration_list_min
df['Classification'] = cls_list
# Criando uma nova coluna para representar a década
df['Ratings'] = df['Ratings'].astype(float)
df['Year'] = df['Year'].astype(int)
df['Decade'] = df['Year'] // 10 * 10
df

In [ ]:
df.to_csv("imdb.csv")

## Análise dos dados

### Análise iniciais
Os resultados iniciais incluem as seguintes estatísticas nas colunas numéricas: média, mínimo e máximo.

In [ ]:
media_notas = df['Ratings'].mean()
print("Média da coluna 'Ratings':", media_notas)

menor_nota = df['Ratings'].min()
print("Menor valor na coluna 'Ratings':", menor_nota)

maior_nota = df['Ratings'].max()
print("Maior valor na coluna 'Ratings':", maior_nota)

In [ ]:
media_minutos = df['Duration (Minutes)'].mean()
print("Média da coluna 'Duration (Minutes)':", media_minutos)

menor_minutos = df['Duration (Minutes)'].min()
print("Menor valor na coluna 'Duration (Minutes)':", menor_minutos)

maior_minutos = df['Duration (Minutes)'].max()
print("Maior valor na coluna 'Duration (Minutes)':", maior_minutos)

In [ ]:
media_ano = df['Year'].mean()
print("Média da coluna 'Year':", media_ano)

menor_ano = df['Year'].min()
print("Menor valor na coluna 'Year':", menor_ano)

maior_ano = df['Year'].max()
print("Maior valor na coluna 'Year':", maior_ano)

### Análises Aprofundadas
 Aqui é realizada análises com gráficos e números.

#### Quantidade de filmes por Década

In [ ]:
contagem_decadas = df['Decade'].value_counts()
contagem_decadas = contagem_decadas.sort_index()
contagem_decadas.plot(kind='bar')

plt.xlabel('Anos')
plt.ylabel('Quantidade')
plt.title('Quantidade de filmes por década')
plt.show()

#### Contagem anual

In [ ]:
contagem_filmes_por_ano = df['Year'].value_counts().sort_index()


plt.hist(df['Year'], bins=20, edgecolor='black')
plt.title('Histograma da Contagem de Filmes por Ano')
plt.xlabel('Ano de Lançamento')
plt.ylabel('Contagem de Filmes')
plt.xticks(rotation=45)
plt.show()

In [ ]:
contagem_filmes_por_ano = df['Year'].value_counts()

print(contagem_filmes_por_ano)

In [ ]:
contagem_filmes_por_ano = df['Year'].value_counts()


ano_com_mais_filmes = contagem_filmes_por_ano.idxmax()
quantidade_filmes = contagem_filmes_por_ano.max()

print("Ano com mais filmes:", ano_com_mais_filmes)
print("Quantidade de filmes:", quantidade_filmes)

In [ ]:
filmes_de_1995 = df[df['Year'] == 1995]

print(filmes_de_1995)

Os gráficos acima demonstram que, a partir dos anos 80, houve uma produção significativa de grandes filmes, como "Star Wars" e "De Volta para o Futuro". Um ponto interessante a mencionar é que os anos 2000 foram a década em que foram lançados os melhores filmes, de acordo com o IMDB.

Além disso, 1995 foi o ano com mais filmes de sucesso, incluindo "Se7en" e "Toy Story".

#### Duração dos filmes em minutos

In [ ]:
plt.hist(df['Duration (Minutes)'], bins=20, edgecolor='black')
plt.title('Histograma da Duração dos Filmes')
plt.xlabel('Duração (minutos)')
plt.ylabel('Quantidade')
plt.show()

A maioria dos filmes tem entre 100 a 150 minutos de duração.

#### Notas de Avaliação

In [ ]:
plt.hist(df['Ratings'], bins=20, edgecolor='black')
plt.title('Histograma das Avaliações dos Filmes')
plt.xlabel('Avaliações')
plt.ylabel('Frequência')
plt.show()

#### Classificação

In [ ]:
contagem_classification = df['Classification'].value_counts().sort_index()
contagem_classification.plot(kind='barh', color=sns.color_palette('Dark2'))
plt.title('Contagem de Filmes por Classificação')
plt.xlabel('Quantidade de Filmes')
plt.ylabel('Classificação')
plt.gca().spines[['top', 'right']].set_visible(False)
plt.grid(axis='x')

plt.show()

#### Década x Notas de avaliações

In [ ]:
plt.scatter(df['Ratings'], df['Year'], alpha=0.5)
plt.title('Avaliações de Filmes ao Longo do Tempo')
plt.xlabel('Notas de Avaliações')
plt.ylabel('Ano de Lançamento')
plt.grid(True)
plt.show()

In [ ]:
media_notas_por_decada = df.groupby('Decade')['Ratings'].mean()

print("Média das notas por década:")
print(media_notas_por_decada)



In [ ]:
contagem_filmes_por_decada = df['Decade'].value_counts()

print(contagem_filmes_por_decada)


In [ ]:
decadas_com_10_filmes_ou_mais = contagem_filmes_por_decada[contagem_filmes_por_decada >= 10].index

df_filtrado = df[df['Decade'].isin(decadas_com_10_filmes_ou_mais)]

media_avaliacoes_por_decada = df_filtrado.groupby('Decade')['Ratings'].mean()

print(media_avaliacoes_por_decada)

Foi realizada uma análise para determinar qual década obteve a maior média de avaliações de filmes. Na primeira análise, o resultado apontou para os anos 2020. No entanto, essa década apresenta uma pequena quantidade de filmes. Para corrigir essa discrepância, aplicou-se um filtro que elimina décadas com menos de 10 filmes. Com esse ajuste, a década de 90 apresenta a maior média, com uma nota de 8.38.

#### Notas de avaliação x Duração

In [ ]:
plt.scatter(df['Ratings'], df['Duration (Minutes)'], alpha=0.5)
plt.title('Notas de Avaliação vs. Duração dos Filmes')
plt.xlabel('Notas de Avaliação')
plt.ylabel('Duração (minutos)')
plt.grid(True)
plt.show()

O gráfico acima mostra que a maioria dos filmes têm entre 100 e 200 minutos de duração, indicando que não é necessário um filme ser muito longo para alcançar sucesso.

#### Duração x Ano




In [ ]:
plt.scatter(df['Year'], df['Duration (Minutes)'], alpha=0.5)
plt.title('Duração dos Filmes vs. Ano')
plt.xlabel('Ano')
plt.ylabel('Duração (minutos)')
plt.grid(True)
plt.show()

Os filmes mais antigos, especialmente nos anos 20, tinham uma duração mais curta. A partir dos anos 40, os filmes começaram a apresentar a duração média das obras atuais.

#### Classificação x Década

In [ ]:
contagem_classificacao_ano = df.groupby(['Decade', 'Classification']).size().unstack(fill_value=0)

plt.figure(figsize=(14, 8))
contagem_classificacao_ano.plot(kind='bar', stacked=True, colormap='Dark2', edgecolor='black')

plt.title('Quantidade de Filmes por Classificação ao Longo das Décadas')
plt.xlabel('Década')
plt.ylabel('Quantidade de Filmes')
plt.legend(title='Classificação')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=45)
plt.gca().spines[['top', 'right']].set_visible(False)

plt.show()

O gráfico mostra que, dos anos 20 aos anos 60, a maioria dos filmes era mais acessível ao público geral. No entanto, os filmes mais recentes têm uma classificação mais restrita (R), que no Brasil equivale à classificação para maiores de 16 anos.

#### Classificação x Notas de avaliação

In [ ]:
contagem_classificacao_ano = df.groupby(['Ratings', 'Classification']).size().unstack(fill_value=0)

plt.figure(figsize=(14, 8))
contagem_classificacao_ano.plot(kind='bar', stacked=True, colormap='Dark2', edgecolor='black')

plt.title('Quantidade de Filmes por Classificação de acordo com as Notas de Avaliação')
plt.xlabel('Notas de Avaliação')
plt.ylabel('Quantidade de Filmes')
plt.legend(title='Classificação')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=45)
plt.gca().spines[['top', 'right']].set_visible(False)

plt.show()

Podemos perceber que os filmes classificados como 'R' são os mais bem-sucedidos por conta da sua quantiade e também por notas de avaliação.

#### Classificação x Duração

In [ ]:
bins = [0, 60, 90, 120, 150, 180, float('inf')]
labels = ['<60', '60-90', '90-120', '120-150', '150-180', '180+']
df['Duration Category'] = pd.cut(df['Duration (Minutes)'], bins=bins, labels=labels)

# Calcular a contagem de filmes por faixa de duração e classificação
contagem_classificacao_duracao = df.groupby(['Duration Category', 'Classification']).size().unstack(fill_value=0)

plt.figure(figsize=(14, 8))
contagem_classificacao_duracao.plot(kind='bar', stacked=True, colormap='Dark2', edgecolor='black')

plt.title('Quantidade de Filmes por Classificação de acordo com a Duração dos Filmes')
plt.xlabel('Duração (Minutos)')
plt.ylabel('Quantidade de Filmes')
plt.legend(title='Classificação')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=45)
plt.gca().spines[['top', 'right']].set_visible(False)

plt.show()

O gráfico demonstra que filmes com duração superior a 90 minutos têm mais classificações 'R', ou seja, são mais restritos em termos de faixa etária.

#### Curisosidades


##### Filme mais longo

In [ ]:
filme_mais_longo = df[df['Duration (Minutes)'] == df['Duration (Minutes)'].max()]
print("Filme mais longo:")
print("Title:", filme_mais_longo['Title'].iloc[0])
print("Duração:", filme_mais_longo['Duration (Minutes)'].iloc[0], "minutos")


##### Filme mais curto

In [ ]:
filme_mais_curto = df[df['Duration (Minutes)'] == df['Duration (Minutes)'].min()]
print("Filme mais curto:")
print("Title:", filme_mais_curto['Title'].iloc[0])
print("Duração:", filme_mais_curto['Duration (Minutes)'].iloc[0], "minutos")


##### Filme com a maior avaliação

In [ ]:
filme_melhor_avaliado = df[df['Ratings'] == df['Ratings'].max()]
print("Filme melhor avaliado:")
print("Title:", filme_melhor_avaliado['Title'].iloc[0])
print("Avaliação:", filme_melhor_avaliado['Ratings'].iloc[0])


Filme com a pior avaliação

In [ ]:
filme_pior_avaliado = df[df['Ratings'] == df['Ratings'].min()]
print("Filme pior avaliado:")
print("Title:", filme_pior_avaliado['Title'].iloc[0])
print("Avaliação:", filme_pior_avaliado['Ratings'].iloc[0])


##### Filme mais antigo

In [ ]:
filme_mais_antigo = df[df['Year'] == df['Year'].min()]
print("Filme mais antigo:")
print("Title:", filme_mais_antigo['Title'].iloc[0])
print("Ano:", filme_mais_antigo['Year'].iloc[0])


##### Filme mais novo

In [ ]:
filme_mais_novo = df[df['Year'] == df['Year'].max()]
print("Filme mais novo:")
print("Title:", filme_mais_novo['Title'].iloc[0])
print("Ano:", filme_mais_novo['Year'].iloc[0])
